# Interrupt Interval Analysis

In [157]:
import logging
from conf import LisaLogging
LisaLogging.setup(level=100)

In [158]:
%matplotlib inline

import os

# Support to access the remote target
from env import TestEnv

# Support to access cpuidle information from the target
from devlib import *

# Support to configure and run RTApp based workloads
from wlgen import RTA, Ramp

# Support for trace events analysis
from trace import Trace
from trace import ResidencyTime, ResidencyData

# DataFrame support
import pandas as pd
from pandas import DataFrame

# Trappy (plots) support
from trappy import ILinePlot
from trappy.stats.grammar import Parser

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

# Hikey960 Platform Configurations

In [159]:
# Setup target configuration
hikey960_conf = {
    "platform"     : 'generic_linux',
    "board"        : 'hikey960',
    "results_dir"  : "test",
    "modules"      : [ 'cpufreq', 'cpuidle' ],
    "ftrace"       : {
        "events" : [ "irq_handler_entry" ],
        "buffsize" : 100 * 1024,
    },
    "tools"        : [ 'trace-cmd', 'taskset', 'rt-app', 'sysbench' ],
}

In [160]:
hikey960_platform = {
    'kernel': {
        'major': 4,
        'sha1': '78e8b90',
        'version': '57 SMP PREEMPT Sat Jul 1 15:42:00 CST 2017',
        'version_number': 14,
        'parts': [4, 14, 74],
        'rc': None,
        'release': '4.4.74-g78e8b90',
        'minor': 74
    },
    
    'freqs': {
        'big':    [ 533000,  999000, 1402000, 1709000, 1844000 ],
        'little': [ 903000, 1421000, 1805000, 2112000, 2362000 ]
    },
    
    'nrg_model': {
        'big': {
            'cluster': {
                'nrg_max': 433
            },
            'cpu': {
                'cap_max': 1024,
                'nrg_max': 2848
            }
        },
        'little': {
            'cluster': {
                'nrg_max': 144
            },
            'cpu': {
                'cap_max': 462,
                'nrg_max': 502
            }
        }
    },
    
    'clusters': {
        'big'    : [4, 5, 6, 7],
        'little' : [0, 1, 2, 3]
    },
    'cpus_count': 8,
    'topology': [[0, 1, 2, 3], [4, 5, 6, 7]]
}

# Interrupt interval statistics

# Idle state estimation statistics (Check if break residency)

In [171]:
from collections import namedtuple

def dfIrqStatistics(irq_data, cpus=None):

    if cpus is None:
        # Generate plots only for available CPUs
        _cpus = range(irq_data.__cpu.max() + 1)
    else:
        _cpus = listify(cpus)

    # Generate valid interrupt events
    available_irqs = sorted(irq_data.irq.unique())
    
    for cpu in _cpus:
        for irq in available_irqs:
            df = irq_data[(irq_data.__cpu == cpu) & (irq_data.irq == irq)]
            
            if df.empty:
                continue
                
            #print df.head(5)
            
            df['time'] = df.index
            df['delta'] = (df['time']-df['time'].shift()).fillna(0).round(6) * 1000000
            
            print df
            df.to_csv('irq_stat_cpu' + str(cpu) + '_irq' + str(irq) + '_' + df['name'].iloc[0] + '.csv',
                      columns = ['delta'], index=False, float_format="%.0f")
            #print df
        
    #print available_irqs

## Read trace data

In [172]:
res_dir = '/tmp/'
trace_file = os.path.join(res_dir, "trace.dat")
trace = Trace(hikey960_platform, trace_file, events=hikey960_conf['ftrace']['events'], normalize_time=True)

## Plot idle states residency comparison

In [ ]:
irq_df = trace._dfg_trace_event('irq_handler_entry')
irq_df.head(10)

,__comm,__cpu,__line,__pid,irq,name
Time,,,,,,
0.000506,<idle>,6,110,0,3,arch_timer
0.002643,logd.writer,2,619,306,3,arch_timer
0.002650,Binder_1,3,621,2909,3,arch_timer
0.007830,<idle>,3,1945,0,3,arch_timer
0.011937,logd.reader.per,1,2615,553,3,arch_timer
0.012329,ispout#6^,1,2644,3187,3,arch_timer
0.012641,ispout#6^,1,2647,3187,3,arch_timer
0.012643,<idle>,3,2648,0,3,arch_timer
0.012729,ispout#6^,1,2709,3187,3,arch_timer


In [ ]:
dfIrqStatistics(irq_df)

/home/leoy/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/leoy/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


                    __comm  __cpu   __line  __pid  irq        name       time  \
Time                                                                            
7.116999             <...>      0  1583903     86    3  arch_timer   7.116999   
7.120848            <idle>      0  1585690      0    3  arch_timer   7.120848   
7.122645            <idle>      0  1585990      0    3  arch_timer   7.122645   
7.132643            <idle>      0  1588233      0    3  arch_timer   7.132643   
7.135616             <...>      0  1588942   3158    3  arch_timer   7.135616   
7.136884            <idle>      0  1589378      0    3  arch_timer   7.136884   
7.142644            <idle>      0  1592448      0    3  arch_timer   7.142644   
7.152643            <idle>      0  1596094      0    3  arch_timer   7.152643   
7.156899            <idle>      0  1597348      0    3  arch_timer   7.156899   
7.161255            <idle>      0  1599259      0    3  arch_timer   7.161255   
7.162641            <idle>  

             __comm  __cpu   __line  __pid  irq     name       time  delta
Time                                                                      
24.615733  Binder_1      0  5922342    417    9  ipc_irq  24.615733    0.0
                    __comm  __cpu   __line  __pid  irq          name  \
Time                                                                   
7.113700            <idle>      0  1582998      0   40  fdf0d000.i2c   
7.113840             <...>      0  1583011     86   40  fdf0d000.i2c   
7.114086             <...>      0  1583024   3158   40  fdf0d000.i2c   
7.114192             <...>      0  1583051     86   40  fdf0d000.i2c   
7.114370             <...>      0  1583064   3158   40  fdf0d000.i2c   
7.114469             <...>      0  1583095     86   40  fdf0d000.i2c   
7.114659             <...>      0  1583123   3158   40  fdf0d000.i2c   
7.114760             <...>      0  1583139     86   40  fdf0d000.i2c   
7.114862            <idle>      0  1583168      0   40 

                    __comm  __cpu   __line  __pid  irq     name       time  \
Time                                                                         
7.140084            <idle>      0  1591041      0   53  gpufreq   7.140084   
7.140218            <idle>      0  1591156      0   53  gpufreq   7.140218   
7.140441            <idle>      0  1591458      0   53  gpufreq   7.140441   
7.143852            <idle>      0  1593320      0   53  gpufreq   7.143852   
7.157108            <idle>      0  1597499      0   53  gpufreq   7.157108   
7.160603            <idle>      0  1599127      0   53  gpufreq   7.160603   
7.189556            <idle>      0  1606761      0   53  gpufreq   7.189556   
7.193054            <idle>      0  1608430      0   53  gpufreq   7.193054   
7.240591              Fill      0  1619270   3151   53  gpufreq   7.240591   
7.244361            <idle>      0  1620944      0   53  gpufreq   7.244361   
7.258067              Fill      0  1625428   3151   53  gpufreq 

                    __comm  __cpu   __line  __pid  irq     name       time  \
Time                                                                         
7.125051            <idle>      0  1586980      0   56  irq_pdp   7.125051   
7.126327            <idle>      0  1587308      0   56  irq_pdp   7.126327   
7.143188            <idle>      0  1592866      0   56  irq_pdp   7.143188   
7.160056            <idle>      0  1599064      0   56  irq_pdp   7.160056   
7.175696            <idle>      0  1601226      0   56  irq_pdp   7.175696   
7.176966            <idle>      0  1601910      0   56  irq_pdp   7.176966   
7.192609            <idle>      0  1607723      0   56  irq_pdp   7.192609   
7.193879            <idle>      0  1608910      0   56  irq_pdp   7.193879   
7.210746            <idle>      0  1612071      0   56  irq_pdp   7.210746   
7.226387            <idle>      0  1617195      0   56  irq_pdp   7.226387   
7.227657            <idle>      0  1617199      0   56  irq_pdp 

                    __comm  __cpu   __line  __pid  irq        name       time  \
Time                                                                            
7.362800            <idle>      0  1654508      0   65  mailbox-10   7.362800   
7.363049            <idle>      0  1654667      0   65  mailbox-10   7.363049   
7.363174       ksoftirqd/0      0  1654726      3   65  mailbox-10   7.363174   
7.363302             <...>      0  1654804     86   65  mailbox-10   7.363302   
7.363396            <idle>      0  1654846      0   65  mailbox-10   7.363396   
7.363475            <idle>      0  1654893      0   65  mailbox-10   7.363475   
7.612842             <...>      0  1725909   3230   65  mailbox-10   7.612842   
7.612933             <...>      0  1725963   3230   65  mailbox-10   7.612933   
7.613005             <...>      0  1725994   3230   65  mailbox-10   7.613005   
7.613084             <...>      0  1726025   3230   65  mailbox-10   7.613084   
7.613215             <...>  

                    __comm  __cpu   __line  __pid  irq       name       time  \
Time                                                                           
7.119101            <idle>      0  1584916      0   67  mailbox-0   7.119101   
7.192860            <idle>      0  1608164      0   67  mailbox-0   7.192860   
7.304139            <idle>      0  1637116      0   67  mailbox-0   7.304139   
7.363076            <idle>      0  1654670      0   67  mailbox-0   7.363076   
7.412844           mmcqd/0      0  1667633    191   67  mailbox-0   7.412844   
7.522837             <...>      0  1701830   3230   67  mailbox-0   7.522837   
7.632322             <...>      0  1731942   3230   67  mailbox-0   7.632322   
7.633417             <...>      0  1732377   3230   67  mailbox-0   7.633417   
7.736390            <idle>      0  1765907      0   67  mailbox-0   7.736390   
7.742855            <idle>      0  1768828      0   67  mailbox-0   7.742855   
7.842857            <idle>      0  17953

                    __comm  __cpu   __line  __pid  irq       name       time  \
Time                                                                           
7.117826             <...>      0  1584551     86   71  mailbox-4   7.117826   
7.118046            <idle>      0  1584602      0   71  mailbox-4   7.118046   
7.119688        glgps47531      0  1585352    617   71  mailbox-4   7.119688   
7.123133            <idle>      0  1586222      0   71  mailbox-4   7.123133   
7.127804            <idle>      0  1588053      0   71  mailbox-4   7.127804   
7.128024     SensorService      0  1588076    991   71  mailbox-4   7.128024   
7.129655            <idle>      0  1588157      0   71  mailbox-4   7.129655   
7.129770            <idle>      0  1588183      0   71  mailbox-4   7.129770   
7.133113            <idle>      0  1588322      0   71  mailbox-4   7.133113   
7.137813            <idle>      0  1589891      0   71  mailbox-4   7.137813   
7.138035     SensorService      0  15901